# SciTeX Statistics - Comprehensive Statistical Analysis

This notebook demonstrates the statistical analysis capabilities of SciTeX's stats module.

The `scitex.stats` module provides:
- Correlation and statistical tests
- Multiple comparison corrections
- Descriptive statistics
- P-value formatting and interpretation
- Advanced statistical workflows

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scitex as stx
from scipy import stats
import seaborn as sns

# Setup
np.random.seed(42)
plt.style.use('default')
print(f"SciTeX version: {stx.__version__}")

## 1. Correlation Analysis

SciTeX provides enhanced correlation testing with proper statistics reporting.

In [ ]:
# Generate correlated data with different relationships
n_samples = 100
x1 = np.random.randn(n_samples)
x2 = 0.8 * x1 + 0.6 * np.random.randn(n_samples)  # Strong positive correlation
x3 = -0.6 * x1 + 0.8 * np.random.randn(n_samples)  # Moderate negative correlation
x4 = np.random.randn(n_samples)  # No correlation

# Test correlations using SciTeX
corr_results = {}
correlations = [
    ('x1 vs x2', x1, x2),
    ('x1 vs x3', x1, x3),
    ('x1 vs x4', x1, x4),
    ('x2 vs x3', x2, x3)
]

print("Correlation Analysis Results:")
print("=" * 50)

for name, var1, var2 in correlations:
    result = stx.stats.corr_test(var1, var2)
    stars = stx.stats.p2stars(result['p'])
    corr_results[name] = result
    
    print(f"{name:12} | r = {result['r']:6.3f} | p = {result['p']:7.4f} | {stars if stars else 'ns':>3}")

print("\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05")

In [ ]:
# Visualize correlations
fig, axes = stx.plt.subplots(2, 2, figsize=(10, 8))
variables = [('x1', 'x2', x1, x2), ('x1', 'x3', x1, x3), ('x1', 'x4', x1, x4), ('x2', 'x3', x2, x3)]

for ax, (name1, name2, var1, var2) in zip(axes.flat, variables):
    # Scatter plot with regression line
    ax.scatter(var1, var2, alpha=0.6, s=30)
    
    # Add regression line
    z = np.polyfit(var1, var2, 1)
    p = np.poly1d(z)
    ax.plot(sorted(var1), p(sorted(var1)), "r--", alpha=0.8)
    
    # Get correlation result
    key = f'{name1} vs {name2}'
    result = corr_results[key]
    stars = stx.stats.p2stars(result['p'])
    
    ax.set_xyt(name1, name2, f"r = {result['r']:.3f} {stars if stars else 'ns'}")
    ax.grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/correlation_analysis.png", symlink_from_cwd=True)
plt.show()

## 2. Multiple Comparison Corrections

When performing multiple statistical tests, correction for multiple comparisons is essential.

In [ ]:
# Generate multiple groups for comparison
n_groups = 6
n_per_group = 30

# Create groups with different means (some significant, some not)
groups = {
    'Group A': np.random.normal(0, 1, n_per_group),
    'Group B': np.random.normal(0.2, 1, n_per_group),   # Small effect
    'Group C': np.random.normal(0.8, 1, n_per_group),   # Medium effect
    'Group D': np.random.normal(1.2, 1, n_per_group),   # Large effect
    'Group E': np.random.normal(0.1, 1, n_per_group),   # Very small effect
    'Group F': np.random.normal(-0.3, 1, n_per_group),  # Small negative effect
}

# Perform all pairwise comparisons
group_names = list(groups.keys())
p_values = []
comparisons = []
effect_sizes = []

for i in range(len(group_names)):
    for j in range(i+1, len(group_names)):
        group1, group2 = group_names[i], group_names[j]
        data1, data2 = groups[group1], groups[group2]
        
        # T-test
        t_stat, p_val = stats.ttest_ind(data1, data2)
        
        # Effect size (Cohen's d)
        pooled_std = np.sqrt((np.var(data1) + np.var(data2)) / 2)
        cohens_d = (np.mean(data2) - np.mean(data1)) / pooled_std
        
        comparisons.append(f"{group1} vs {group2}")
        p_values.append(p_val)
        effect_sizes.append(cohens_d)

# Apply multiple comparison corrections
p_bonferroni = stx.stats.bonferroni_correction(p_values)
p_fdr = stx.stats.fdr_correction(p_values)

print(f"Number of comparisons: {len(comparisons)}")
print(f"\nMultiple Comparison Results:")
print("=" * 80)

In [ ]:
# Create comprehensive results table
results_df = pd.DataFrame({
    'Comparison': comparisons,
    'Effect_Size': effect_sizes,
    'P_uncorrected': p_values,
    'P_Bonferroni': p_bonferroni,
    'P_FDR': p_fdr,
})

# Add significance markers
results_df['Sig_Uncorrected'] = [stx.stats.p2stars(p) if p < 0.05 else 'ns' for p in p_values]
results_df['Sig_Bonferroni'] = [stx.stats.p2stars(p) if p < 0.05 else 'ns' for p in p_bonferroni]
results_df['Sig_FDR'] = [stx.stats.p2stars(p) if p < 0.05 else 'ns' for p in p_fdr]

# Display results
print("Statistical Comparison Results:")
print(results_df.round(4))

# Count significant results
sig_uncorrected = sum([p < 0.05 for p in p_values])
sig_bonferroni = sum([p < 0.05 for p in p_bonferroni])
sig_fdr = sum([p < 0.05 for p in p_fdr])

print(f"\nSignificant results:")
print(f"Uncorrected: {sig_uncorrected}/{len(p_values)}")
print(f"Bonferroni:  {sig_bonferroni}/{len(p_values)}")
print(f"FDR:         {sig_fdr}/{len(p_values)}")

In [ ]:
# Visualize multiple comparison results
fig, axes = stx.plt.subplots(2, 2, figsize=(12, 10))

# Group distributions
ax1 = axes[0, 0]
positions = list(range(len(groups)))
group_data = [groups[name] for name in group_names]

bp = ax1.boxplot(group_data, positions=positions, labels=group_names, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

ax1.set_xyt("Groups", "Values", "Group Distributions")
ax1.grid(True, alpha=0.3)

# P-value comparison
ax2 = axes[0, 1]
x_pos = np.arange(len(comparisons))
ax2.scatter(x_pos, -np.log10(p_values), label='Uncorrected', alpha=0.7)
ax2.scatter(x_pos, -np.log10(p_bonferroni), label='Bonferroni', alpha=0.7)
ax2.scatter(x_pos, -np.log10(p_fdr), label='FDR', alpha=0.7)
ax2.axhline(-np.log10(0.05), color='red', linestyle='--', alpha=0.7, label='p=0.05')
ax2.set_xyt("Comparison Index", "-log10(p-value)", "P-value Corrections")
ax2.legend()
ax2.grid(True, alpha=0.3)

# Effect sizes
ax3 = axes[1, 0]
colors = ['red' if abs(d) > 0.8 else 'orange' if abs(d) > 0.5 else 'green' for d in effect_sizes]
bars = ax3.bar(range(len(effect_sizes)), effect_sizes, color=colors, alpha=0.7)
ax3.axhline(0, color='black', linewidth=0.8)
ax3.set_xyt("Comparison Index", "Cohen's d", "Effect Sizes")
ax3.set_xticks(range(len(comparisons)))
ax3.set_xticklabels([c.replace(' vs ', '\nvs\n') for c in comparisons], rotation=45, ha='right')
ax3.grid(True, alpha=0.3)

# Significance heatmap
ax4 = axes[1, 1]
sig_matrix = np.array([
    [1 if sig != 'ns' else 0 for sig in results_df['Sig_Uncorrected']],
    [1 if sig != 'ns' else 0 for sig in results_df['Sig_Bonferroni']],
    [1 if sig != 'ns' else 0 for sig in results_df['Sig_FDR']]
])

im = ax4.imshow(sig_matrix, cmap='RdYlBu_r', aspect='auto')
ax4.set_xyt("Comparison Index", "", "Significance Matrix")
ax4.set_yticks([0, 1, 2])
ax4.set_yticklabels(['Uncorrected', 'Bonferroni', 'FDR'])
ax4.set_xticks(range(len(comparisons)))
ax4.set_xticklabels(range(len(comparisons)))

plt.tight_layout()
stx.io.save(fig, "./figures/multiple_comparisons.png", symlink_from_cwd=True)
plt.show()

## 3. Descriptive Statistics

Comprehensive descriptive analysis with SciTeX utilities.

In [ ]:
# Generate realistic dataset
n_subjects = 200
np.random.seed(123)

# Simulate experimental data
data = {
    'age': np.random.normal(35, 10, n_subjects).clip(18, 80),
    'reaction_time': np.random.lognormal(np.log(300), 0.3, n_subjects),  # Skewed
    'accuracy': np.random.beta(8, 2, n_subjects),  # Bounded [0,1]
    'score': np.random.normal(75, 15, n_subjects).clip(0, 100),
    'group': np.random.choice(['Control', 'Treatment'], n_subjects),
    'session': np.random.choice(['Session1', 'Session2', 'Session3'], n_subjects)
}

# Convert to DataFrame
df = stx.pd.force_df(data)

# Add some missing values to make it realistic
missing_idx = np.random.choice(n_subjects, 10, replace=False)
df.loc[missing_idx, 'reaction_time'] = np.nan

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nMissing values:")
print(df.isnull().sum())

In [ ]:
# Comprehensive descriptive statistics
numeric_vars = ['age', 'reaction_time', 'accuracy', 'score']

print("Descriptive Statistics Summary:")
print("=" * 70)

desc_stats = {}
for var in numeric_vars:
    data_clean = df[var].dropna()
    
    stats_dict = {
        'count': len(data_clean),
        'mean': np.mean(data_clean),
        'std': np.std(data_clean, ddof=1),
        'min': np.min(data_clean),
        'q25': np.percentile(data_clean, 25),
        'median': np.median(data_clean),
        'q75': np.percentile(data_clean, 75),
        'max': np.max(data_clean),
        'skewness': stats.skew(data_clean),
        'kurtosis': stats.kurtosis(data_clean)
    }
    
    desc_stats[var] = stats_dict
    
    print(f"\n{var.upper()}:")
    print(f"  N = {stats_dict['count']}, Mean = {stats_dict['mean']:.2f} ± {stats_dict['std']:.2f}")
    print(f"  Median = {stats_dict['median']:.2f} [Q1={stats_dict['q25']:.2f}, Q3={stats_dict['q75']:.2f}]")
    print(f"  Range = [{stats_dict['min']:.2f}, {stats_dict['max']:.2f}]")
    print(f"  Skewness = {stats_dict['skewness']:.3f}, Kurtosis = {stats_dict['kurtosis']:.3f}")

# Convert to DataFrame for easier handling
desc_df = stx.pd.force_df(desc_stats).T
print(f"\nComplete descriptive statistics table:")
print(desc_df.round(3))

In [ ]:
# Visualize distributions and descriptive statistics
fig, axes = stx.plt.subplots(2, 4, figsize=(16, 8))

for i, var in enumerate(numeric_vars):
    data_clean = df[var].dropna()
    
    # Histogram with normal curve overlay
    ax1 = axes[0, i]
    n, bins, patches = ax1.hist(data_clean, bins=20, alpha=0.7, density=True, color='skyblue')
    
    # Overlay normal distribution
    mu, sigma = np.mean(data_clean), np.std(data_clean)
    x = np.linspace(data_clean.min(), data_clean.max(), 100)
    ax1.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal fit')
    
    ax1.axvline(mu, color='red', linestyle='--', alpha=0.7, label=f'Mean={mu:.1f}')
    ax1.axvline(np.median(data_clean), color='orange', linestyle='--', alpha=0.7, label=f'Median={np.median(data_clean):.1f}')
    
    ax1.set_xyt(var.replace('_', ' ').title(), 'Density', f'{var.title()} Distribution')
    ax1.legend(fontsize=8)
    ax1.grid(True, alpha=0.3)
    
    # Q-Q plot for normality assessment
    ax2 = axes[1, i]
    stats.probplot(data_clean, dist="norm", plot=ax2)
    ax2.set_xyt('Theoretical Quantiles', 'Sample Quantiles', f'{var.title()} Q-Q Plot')
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/descriptive_statistics.png", symlink_from_cwd=True)
plt.show()

## 4. Group Comparisons and ANOVA

Comparing groups with proper effect size reporting.

In [ ]:
# Compare groups using various statistical tests
print("Group Comparison Analysis:")
print("=" * 50)

# Two-group comparisons
for var in ['reaction_time', 'accuracy', 'score']:
    control_data = df[df['group'] == 'Control'][var].dropna()
    treatment_data = df[df['group'] == 'Treatment'][var].dropna()
    
    # T-test
    t_stat, p_ttest = stats.ttest_ind(control_data, treatment_data)
    
    # Mann-Whitney U test (non-parametric)
    u_stat, p_mann = stats.mannwhitneyu(control_data, treatment_data, alternative='two-sided')
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(control_data)-1)*np.var(control_data, ddof=1) + 
                         (len(treatment_data)-1)*np.var(treatment_data, ddof=1)) / 
                        (len(control_data) + len(treatment_data) - 2))
    cohens_d = (np.mean(treatment_data) - np.mean(control_data)) / pooled_std
    
    print(f"\n{var.upper()}:")
    print(f"  Control:   {np.mean(control_data):.2f} ± {np.std(control_data, ddof=1):.2f} (n={len(control_data)})")
    print(f"  Treatment: {np.mean(treatment_data):.2f} ± {np.std(treatment_data, ddof=1):.2f} (n={len(treatment_data)})")
    print(f"  T-test:    t = {t_stat:.3f}, p = {p_ttest:.4f} {stx.stats.p2stars(p_ttest)}")
    print(f"  Mann-Whitney: U = {u_stat:.1f}, p = {p_mann:.4f} {stx.stats.p2stars(p_mann)}")
    print(f"  Effect size: Cohen's d = {cohens_d:.3f}")
    
    # Interpret effect size
    if abs(cohens_d) < 0.2:
        effect_interp = "negligible"
    elif abs(cohens_d) < 0.5:
        effect_interp = "small"
    elif abs(cohens_d) < 0.8:
        effect_interp = "medium"
    else:
        effect_interp = "large"
    
    print(f"  Interpretation: {effect_interp} effect")

In [ ]:
# One-way ANOVA for session effects
print("\nOne-way ANOVA - Session Effects:")
print("=" * 40)

for var in ['reaction_time', 'accuracy', 'score']:
    # Prepare data for ANOVA
    session_groups = []
    for session in ['Session1', 'Session2', 'Session3']:
        session_data = df[df['session'] == session][var].dropna()
        session_groups.append(session_data)
    
    # Perform ANOVA
    f_stat, p_anova = stats.f_oneway(*session_groups)
    
    # Calculate eta-squared (effect size for ANOVA)
    all_data = np.concatenate(session_groups)
    grand_mean = np.mean(all_data)
    
    ss_between = sum([len(group) * (np.mean(group) - grand_mean)**2 for group in session_groups])
    ss_total = sum([(x - grand_mean)**2 for x in all_data])
    eta_squared = ss_between / ss_total
    
    print(f"\n{var.upper()}:")
    for i, session in enumerate(['Session1', 'Session2', 'Session3']):
        group_data = session_groups[i]
        print(f"  {session}: {np.mean(group_data):.2f} ± {np.std(group_data, ddof=1):.2f} (n={len(group_data)})")
    
    print(f"  F({len(session_groups)-1}, {len(all_data)-len(session_groups)}) = {f_stat:.3f}")
    print(f"  p = {p_anova:.4f} {stx.stats.p2stars(p_anova)}")
    print(f"  η² = {eta_squared:.3f}")

In [ ]:
# Visualize group comparisons
fig, axes = stx.plt.subplots(2, 3, figsize=(15, 10))

# Group comparisons (Control vs Treatment)
for i, var in enumerate(['reaction_time', 'accuracy', 'score']):
    ax = axes[0, i]
    
    # Box plots
    control_data = df[df['group'] == 'Control'][var].dropna()
    treatment_data = df[df['group'] == 'Treatment'][var].dropna()
    
    bp = ax.boxplot([control_data, treatment_data], 
                    labels=['Control', 'Treatment'],
                    patch_artist=True)
    
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][1].set_facecolor('lightcoral')
    
    # Add statistical annotation
    t_stat, p_val = stats.ttest_ind(control_data, treatment_data)
    stars = stx.stats.p2stars(p_val)
    
    y_max = max(control_data.max(), treatment_data.max())
    y_pos = y_max * 1.1
    ax.plot([1, 2], [y_pos, y_pos], 'k-', linewidth=1)
    ax.text(1.5, y_pos*1.02, f'p = {p_val:.3f} {stars if stars else "ns"}', 
            ha='center', va='bottom')
    
    ax.set_xyt('Group', var.replace('_', ' ').title(), f'{var.title()} by Group')
    ax.grid(True, alpha=0.3)

# Session comparisons (ANOVA)
for i, var in enumerate(['reaction_time', 'accuracy', 'score']):
    ax = axes[1, i]
    
    # Box plots for sessions
    session_data = []
    for session in ['Session1', 'Session2', 'Session3']:
        session_data.append(df[df['session'] == session][var].dropna())
    
    bp = ax.boxplot(session_data, labels=['S1', 'S2', 'S3'], patch_artist=True)
    
    colors = ['lightgreen', 'lightyellow', 'lightpink']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    # Add ANOVA results
    f_stat, p_anova = stats.f_oneway(*session_data)
    stars = stx.stats.p2stars(p_anova)
    
    ax.set_xyt('Session', var.replace('_', ' ').title(), 
               f'{var.title()} by Session\nF = {f_stat:.2f}, p = {p_anova:.3f} {stars if stars else "ns"}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/group_comparisons.png", symlink_from_cwd=True)
plt.show()

## 5. Advanced Statistical Tests

Specialized tests for different data types and distributions.

In [ ]:
# Test for normality
print("Normality Tests:")
print("=" * 30)

for var in numeric_vars:
    data_clean = df[var].dropna()
    
    # Shapiro-Wilk test
    shapiro_stat, shapiro_p = stats.shapiro(data_clean)
    
    # Anderson-Darling test
    ad_stat, ad_crit, ad_sig = stats.anderson(data_clean, dist='norm')
    
    # Kolmogorov-Smirnov test
    ks_stat, ks_p = stats.kstest(data_clean, 'norm', 
                                args=(np.mean(data_clean), np.std(data_clean, ddof=1)))
    
    print(f"\n{var.upper()}:")
    print(f"  Shapiro-Wilk:  W = {shapiro_stat:.4f}, p = {shapiro_p:.4f} {stx.stats.p2stars(shapiro_p)}")
    print(f"  Kolmogorov-S:  D = {ks_stat:.4f}, p = {ks_p:.4f} {stx.stats.p2stars(ks_p)}")
    print(f"  Anderson-D:    A² = {ad_stat:.4f}")
    
    # Interpretation
    if shapiro_p < 0.05:
        norm_interp = "Non-normal distribution"
    else:
        norm_interp = "Approximately normal"
    print(f"  Interpretation: {norm_interp}")

In [ ]:
# Homogeneity of variance tests
print("\nHomogeneity of Variance Tests:")
print("=" * 40)

for var in ['reaction_time', 'accuracy', 'score']:
    # Group data
    control_data = df[df['group'] == 'Control'][var].dropna()
    treatment_data = df[df['group'] == 'Treatment'][var].dropna()
    
    # Levene's test
    levene_stat, levene_p = stats.levene(control_data, treatment_data)
    
    # Bartlett's test (assumes normality)
    bartlett_stat, bartlett_p = stats.bartlett(control_data, treatment_data)
    
    # F-test for equal variances
    f_ratio = np.var(treatment_data, ddof=1) / np.var(control_data, ddof=1)
    df1, df2 = len(treatment_data) - 1, len(control_data) - 1
    f_p = 2 * min(stats.f.cdf(f_ratio, df1, df2), 1 - stats.f.cdf(f_ratio, df1, df2))
    
    print(f"\n{var.upper()}:")
    print(f"  Control variance:   {np.var(control_data, ddof=1):.3f}")
    print(f"  Treatment variance: {np.var(treatment_data, ddof=1):.3f}")
    print(f"  Levene's test:  W = {levene_stat:.4f}, p = {levene_p:.4f} {stx.stats.p2stars(levene_p)}")
    print(f"  Bartlett's test: χ² = {bartlett_stat:.4f}, p = {bartlett_p:.4f} {stx.stats.p2stars(bartlett_p)}")
    print(f"  F-test:         F = {f_ratio:.4f}, p = {f_p:.4f} {stx.stats.p2stars(f_p)}")
    
    if levene_p < 0.05:
        var_interp = "Unequal variances"
    else:
        var_interp = "Equal variances"
    print(f"  Interpretation: {var_interp}")

## 6. Outlier Detection and Robust Statistics

Identifying outliers and using robust statistical methods.

In [ ]:
# Outlier detection using multiple methods
print("Outlier Detection Analysis:")
print("=" * 35)

outlier_results = {}

for var in ['reaction_time', 'accuracy', 'score']:
    data_clean = df[var].dropna()
    
    # Method 1: IQR method
    Q1 = np.percentile(data_clean, 25)
    Q3 = np.percentile(data_clean, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    iqr_outliers = data_clean[(data_clean < lower_bound) | (data_clean > upper_bound)]
    
    # Method 2: Z-score method
    z_scores = np.abs(stats.zscore(data_clean))
    zscore_outliers = data_clean[z_scores > 2.5]
    
    # Method 3: Modified Z-score (using median)
    median = np.median(data_clean)
    mad = np.median(np.abs(data_clean - median))
    modified_z_scores = 0.6745 * (data_clean - median) / mad
    mod_zscore_outliers = data_clean[np.abs(modified_z_scores) > 3.5]
    
    outlier_results[var] = {
        'iqr': len(iqr_outliers),
        'zscore': len(zscore_outliers),
        'mod_zscore': len(mod_zscore_outliers)
    }
    
    print(f"\n{var.upper()} (n={len(data_clean)}):")
    print(f"  IQR method:           {len(iqr_outliers)} outliers ({len(iqr_outliers)/len(data_clean)*100:.1f}%)")
    print(f"  Z-score (|z| > 2.5):  {len(zscore_outliers)} outliers ({len(zscore_outliers)/len(data_clean)*100:.1f}%)")
    print(f"  Modified Z-score:     {len(mod_zscore_outliers)} outliers ({len(mod_zscore_outliers)/len(data_clean)*100:.1f}%)")
    
    # Robust vs classical statistics
    classical_mean = np.mean(data_clean)
    classical_std = np.std(data_clean, ddof=1)
    robust_median = np.median(data_clean)
    robust_mad = 1.4826 * mad  # Scale MAD to estimate std
    
    print(f"  Classical: Mean = {classical_mean:.2f}, SD = {classical_std:.2f}")
    print(f"  Robust:    Median = {robust_median:.2f}, MAD = {robust_mad:.2f}")

In [ ]:
# Visualize outliers
fig, axes = stx.plt.subplots(2, 3, figsize=(15, 8))

for i, var in enumerate(['reaction_time', 'accuracy', 'score']):
    data_clean = df[var].dropna()
    
    # Box plot showing outliers
    ax1 = axes[0, i]
    bp = ax1.boxplot(data_clean, patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    
    # Mark statistical outliers
    Q1 = np.percentile(data_clean, 25)
    Q3 = np.percentile(data_clean, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data_clean[(data_clean < lower_bound) | (data_clean > upper_bound)]
    if len(outliers) > 0:
        ax1.scatter([1] * len(outliers), outliers, color='red', s=30, alpha=0.7)
    
    ax1.set_xyt('', var.replace('_', ' ').title(), f'{var.title()} - Box Plot')
    ax1.set_xticks([])
    ax1.grid(True, alpha=0.3)
    
    # Z-score plot
    ax2 = axes[1, i]
    z_scores = np.abs(stats.zscore(data_clean))
    colors = ['red' if z > 2.5 else 'blue' for z in z_scores]
    ax2.scatter(range(len(z_scores)), z_scores, c=colors, alpha=0.6, s=20)
    ax2.axhline(y=2.5, color='red', linestyle='--', alpha=0.7, label='|z| = 2.5')
    ax2.axhline(y=3, color='orange', linestyle='--', alpha=0.7, label='|z| = 3')
    
    ax2.set_xyt('Sample Index', '|Z-score|', f'{var.title()} - Z-scores')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/outlier_detection.png", symlink_from_cwd=True)
plt.show()

## 7. Statistical Power Analysis

Sample size and power calculations for study design.

In [ ]:
# Power analysis for different effect sizes
from scipy.stats import norm

def power_ttest(n, effect_size, alpha=0.05):
    """Calculate power for two-sample t-test."""
    # Critical t-value
    df = 2 * n - 2
    t_crit = stats.t.ppf(1 - alpha/2, df)
    
    # Non-centrality parameter
    ncp = effect_size * np.sqrt(n/2)
    
    # Power calculation
    power = 1 - stats.nct.cdf(t_crit, df, ncp) + stats.nct.cdf(-t_crit, df, ncp)
    return power

def sample_size_ttest(effect_size, power=0.8, alpha=0.05):
    """Calculate required sample size for desired power."""
    # Approximate sample size calculation
    z_alpha = norm.ppf(1 - alpha/2)
    z_beta = norm.ppf(power)
    n = 2 * ((z_alpha + z_beta) / effect_size) ** 2
    return int(np.ceil(n))

print("Statistical Power Analysis:")
print("=" * 35)

# Effect sizes
effect_sizes = [0.2, 0.5, 0.8, 1.0, 1.2]  # Small, medium, large, very large
sample_sizes = [10, 20, 30, 50, 100, 200]

print("\nPower for different combinations:")
print("Effect Size | Sample Size | Power")
print("-" * 35)

power_matrix = np.zeros((len(effect_sizes), len(sample_sizes)))

for i, es in enumerate(effect_sizes):
    for j, n in enumerate(sample_sizes):
        power = power_ttest(n, es)
        power_matrix[i, j] = power
        if j < 3:  # Print only first few for readability
            print(f"{es:11.1f} | {n:11d} | {power:5.3f}")

print("\nRequired sample sizes for 80% power:")
print("Effect Size | Required N per group")
print("-" * 35)

for es in effect_sizes:
    required_n = sample_size_ttest(es, power=0.8)
    print(f"{es:11.1f} | {required_n:18d}")

In [ ]:
# Visualize power analysis
fig, axes = stx.plt.subplots(1, 2, figsize=(12, 5))

# Power curves
ax1 = axes[0]
for i, es in enumerate(effect_sizes):
    powers = power_matrix[i, :]
    ax1.plot(sample_sizes, powers, 'o-', label=f'd = {es}', linewidth=2, markersize=6)

ax1.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='80% Power')
ax1.set_xyt('Sample Size (per group)', 'Statistical Power', 'Power Curves for T-test')
ax1.set_ylim(0, 1)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Sample size requirements
ax2 = axes[1]
required_ns = [sample_size_ttest(es, power=0.8) for es in effect_sizes]
bars = ax2.bar(range(len(effect_sizes)), required_ns, 
               color=['lightcoral', 'orange', 'lightgreen', 'lightblue', 'purple'],
               alpha=0.7)

# Add value labels on bars
for i, (bar, n) in enumerate(zip(bars, required_ns)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             str(n), ha='center', va='bottom', fontweight='bold')

ax2.set_xyt('Effect Size', 'Required Sample Size\n(per group)', 'Sample Size for 80% Power')
ax2.set_xticks(range(len(effect_sizes)))
ax2.set_xticklabels([f'{es}' for es in effect_sizes])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
stx.io.save(fig, "./figures/power_analysis.png", symlink_from_cwd=True)
plt.show()

## Summary

This notebook demonstrated the comprehensive statistical analysis capabilities of SciTeX:

### Key Features Covered:

1. **Correlation Analysis**: Enhanced correlation testing with significance formatting
2. **Multiple Comparisons**: Bonferroni and FDR corrections for multiple testing
3. **Descriptive Statistics**: Comprehensive summary statistics with distribution analysis
4. **Group Comparisons**: T-tests, Mann-Whitney U, and ANOVA with effect sizes
5. **Advanced Tests**: Normality testing, homogeneity of variance assessment
6. **Outlier Detection**: Multiple methods for identifying and handling outliers
7. **Power Analysis**: Sample size and power calculations for study design

### SciTeX Statistics Advantages:

- **Standardized reporting** with p-value stars and effect sizes
- **Comprehensive test batteries** for assumptions and robustness
- **Integrated visualization** with statistical annotations
- **Publication-ready outputs** with proper statistical formatting
- **Multiple comparison awareness** built into workflows

### Best Practices Demonstrated:

- Always check assumptions (normality, homogeneity of variance)
- Report effect sizes alongside p-values
- Use appropriate corrections for multiple comparisons
- Consider robust alternatives when assumptions are violated
- Perform power analysis for proper study design

### Next Steps:
- Explore `scitex.ai` for machine learning approaches to statistical analysis
- Use `scitex.plt` for advanced statistical visualization
- Check `scitex.pd` for data manipulation and preprocessing

In [ ]:
# Save all statistical results
statistical_summary = {
    'descriptive_statistics': desc_df.to_dict(),
    'correlation_results': {k: v for k, v in corr_results.items()},
    'multiple_comparisons': results_df.to_dict(),
    'outlier_detection': outlier_results,
    'power_analysis': {
        'effect_sizes': effect_sizes,
        'required_sample_sizes': required_ns,
        'power_matrix': power_matrix.tolist()
    },
    'dataset_info': {
        'n_subjects': len(df),
        'variables': list(df.columns),
        'missing_values': df.isnull().sum().to_dict()
    }
}

stx.io.save(statistical_summary, "./data/statistical_analysis_summary.json", symlink_from_cwd=True)
stx.io.save(df, "./data/analysis_dataset.csv", symlink_from_cwd=True)

print("\n✅ Statistical analysis complete!")
print("📊 Results saved to ./data/statistical_analysis_summary.json")
print("📈 Figures saved to ./figures/")
print("🔬 Dataset saved to ./data/analysis_dataset.csv")